In [61]:
# import packages
import requests
import json
import selenium
import pandas as pd
import pickle
import time
import os

In [62]:
# Choose diet
diet = 'mediterranean'

In [63]:
# Get acces API
url = "https://api.edamam.com/api/recipes/v2"

api_id = os.environ['APP_ID']
api_key = os.environ['APP_KEY'] 

headers = {api_id, api_key}

querystring = {"type": "public", "q": diet, "app_id": api_id, "app_key": api_key}

response = requests.get(url, headers={"Authorization": api_key}, params=querystring)

In [64]:
json_response = json.loads(response.text.replace('null', '"None"').replace('True','"True"').replace('False','"False"'))

In [65]:
recipe_request = json_response

In [66]:
still_some_data = True
next_url=url

In [67]:
rec_dic = {}
count = 0 
df_list = []
while still_some_data:
    response = requests.get(next_url, headers={"Authorization": api_key}, params=querystring)
    try: 
      response.json()
    except:
      still_some_data = False
    try:
      recipe_request = response.json()
    except:
      print('headersize: ', print(len(next_url)))
      print(response.text) 
    time.sleep(6)
    
    try:
        print(f'Getting data for '+str(recipe_request['to']))
    except json.decoder.JSONDecodeError:
        print("There was a problem accessing the equipment data.")
        
    if recipe_request['to']==recipe_request['count']:
        still_some_data=False
    if recipe_request['to']> recipe_request['count']:
        still_some_data = False 
        
    next_url = recipe_request['_links']['next']['href']

    
    for idx, recipe in enumerate(recipe_request['hits']):
      count+=1
      if (recipe["recipe"]["mealType"][0]== 'lunch/dinner'):
        # print(recipe["recipe"]["label"])
            str(diet), \
            df_list.append([recipe["recipe"]["label"], \
            str(round(recipe['recipe']["totalNutrients"]['PROCNT']['quantity'], 2))+" g", \
            str(round(recipe['recipe']["totalNutrients"]['CHOCDF']['quantity'], 2))+" g",\
            str(round(recipe['recipe']["totalNutrients"]['FAT']['quantity'], 2))+" g"])

Getting data for 20
Getting data for 40


KeyboardInterrupt: 

In [68]:
print(df_list)
print(len(df_list))

[['Mediterranean Cucumber Tonic', 'mediterranean', '0.25 g', '12.01 g', '0.33 g'], ['Loup de Mer (Mediterranean Seabass)', 'mediterranean', '306.14 g', '43.1 g', '26.55 g'], ['Mediterranean Baked Trout with Olives, Fennel & Tomatoes recipes', 'mediterranean', '187.14 g', '42.31 g', '87.94 g'], ['Mediterranean Salmon Risotto recipes', 'mediterranean', '395.5 g', '287.1 g', '291.04 g'], ['Mediterranean Pita Melts', 'mediterranean', '46.57 g', '139.71 g', '37.7 g'], ['Mediterranean Salmon Stuffed Shells', 'mediterranean', '333.01 g', '414.66 g', '230.05 g'], ['Easy Stuffed Peppers', 'mediterranean', '76.64 g', '429.13 g', '58.98 g'], ['Mediterranean Spiced Cucumber Medallions', 'mediterranean', '29.67 g', '23.24 g', '28.44 g'], ['Marinated Mediterranean Olives', 'mediterranean', '4.76 g', '34.08 g', '50.12 g'], ['Loup De Mer (Mediterranean Seabass)', 'mediterranean', '306.68 g', '45.78 g', '26.54 g'], ['Mediterranean Chicken', 'mediterranean', '281.25 g', '78.76 g', '167.28 g'], ['Grilled

In [30]:
df = pd.DataFrame(df_list, columns = ['Recipe_name', 'Protein', 'Carbs', 'Fat'])

print(df)
print(len(df['Recipe_name'].unique()))

                                Recipe_name   Protein     Carbs       Fat
0                            Keto Fat Bombs   49.19 g   79.78 g   264.4 g
1                     Keto Egg Bites Recipe   92.86 g   16.19 g   86.51 g
2                      Keto Key Lime Mousse   27.49 g   23.58 g  177.43 g
3                Keto Peanut Butter Squares   130.0 g  507.48 g  362.36 g
4     YOGURT RICE, REINVENTED THE KETO WAY!   21.23 g   34.78 g   27.74 g
...                                     ...       ...       ...       ...
1507                    Macadamia Fat Bread   53.77 g   79.92 g  266.07 g
1508   Beef, cream cheese & shredded cheese  169.13 g    7.26 g  239.23 g
1509   No Carb (No Cauliflower) Pizza Crust   225.2 g   10.55 g   94.58 g
1510            Indian Coconut Curry Salmon   62.59 g   39.71 g   65.81 g
1511                                 Eggnog   11.78 g   30.36 g  140.04 g

[1512 rows x 4 columns]
1456


In [31]:
#Save into CSV
df.to_csv('Mediterranean.csv', index = False)

In [32]:
# Or read pandas
pd.read_csv('Mediterranean.csv')

,Recipe_name,Protein,Carbs,Fat
0,Keto Fat Bombs,49.19 g,79.78 g,264.4 g
1,Keto Egg Bites Recipe,92.86 g,16.19 g,86.51 g
2,Keto Key Lime Mousse,27.49 g,23.58 g,177.43 g
3,Keto Peanut Butter Squares,130.0 g,507.48 g,362.36 g
4,"YOGURT RICE, REINVENTED THE KETO WAY!",21.23 g,34.78 g,27.74 g
...,...,...,...,...
1507,Macadamia Fat Bread,53.77 g,79.92 g,266.07 g
1508,"Beef, cream cheese & shredded cheese",169.13 g,7.26 g,239.23 g
1509,No Carb (No Cauliflower) Pizza Crust,225.2 g,10.55 g,94.58 g
1510,Indian Coconut Curry Salmon,62.59 g,39.71 g,65.81 g


In [43]:
# Directory of Keto CSV
'/Users/indi/Documents/Master/Blok 1/ODCM diets/recipe-nutritions/CSV/Keto.csv'

'/Users/indi/Documents/Master/Blok 1/ODCM diets/recipe-nutritions/CSV'

In [47]:
# merging the files
df_all_diets = pd.concat(
    map(pd.read_csv, ['/Users/indi/Documents/Master/Blok 1/ODCM diets/recipe-nutritions/CSV/Keto.csv', '/Users/indi/Documents/Master/Blok 1/ODCM diets/recipe-nutritions/CSV/Dash.csv', '/Users/indi/Documents/Master/Blok 1/ODCM diets/recipe-nutritions/CSV/Vegan.csv']), ignore_index=True)
print(df_all_diets)


                                   Recipe_name  Protein     Carbs       Fat
0                               Keto Fat Bombs  49.19 g   79.78 g   264.4 g
1                        Keto Egg Bites Recipe  92.86 g   16.19 g   86.51 g
2                         Keto Key Lime Mousse  27.49 g   23.58 g  177.43 g
3                   Keto Peanut Butter Squares  130.0 g  507.48 g  362.36 g
4        YOGURT RICE, REINVENTED THE KETO WAY!  21.23 g   34.78 g   27.74 g
...                                        ...      ...       ...       ...
4774  Vegan Parmesan-Crusted Cauliflower Wings  56.28 g   65.62 g   90.77 g
4775                          Vegan Spice Cake   34.4 g  551.66 g  193.42 g
4776                      Fudgy Vegan Brownies  28.65 g  364.77 g  168.68 g
4777           Vegan Spinach Pinwheels recipes  44.84 g   59.62 g   60.33 g
4778                Vegan Noodle Kugel recipes  60.29 g  469.33 g  108.76 g

[4779 rows x 4 columns]


In [57]:
#Save into CSV
df_all_diets.to_csv('All_Diets.csv', index = False)

In [58]:
# Or read pandas
pd.read_csv('All_Diets.csv')

,Recipe_name,Protein,Carbs,Fat
0,Keto Fat Bombs,49.19 g,79.78 g,264.4 g
1,Keto Egg Bites Recipe,92.86 g,16.19 g,86.51 g
2,Keto Key Lime Mousse,27.49 g,23.58 g,177.43 g
3,Keto Peanut Butter Squares,130.0 g,507.48 g,362.36 g
4,"YOGURT RICE, REINVENTED THE KETO WAY!",21.23 g,34.78 g,27.74 g
...,...,...,...,...
4774,Vegan Parmesan-Crusted Cauliflower Wings,56.28 g,65.62 g,90.77 g
4775,Vegan Spice Cake,34.4 g,551.66 g,193.42 g
4776,Fudgy Vegan Brownies,28.65 g,364.77 g,168.68 g
4777,Vegan Spinach Pinwheels recipes,44.84 g,59.62 g,60.33 g
